In [11]:
import pandas as pd
import numpy as np
import datetime as dt
import statsmodels.api as sm
import scipy.stats as stats
import os

import warnings
warnings.filterwarnings("ignore")

# Table 1

In [21]:
def SHIPS_variable_trend(basin, TargetName):       
    ships = pd.read_csv('data/SHIPS/SHIPS_development_%s.csv'%basin)
    ships = ships.replace(9999,np.NaN)
    ships['DATE'] = ships['DATE'].apply(lambda x: str(x).zfill(6))   #fills date to 6 digits with 0
    ships['YEAR'] = ships['DATE'].apply(lambda x: ('19' + str(x)[0:2]) if (str(x)[0]=='8' or str(x)[0]=='9') else ('20' + str(x)[0:2])).astype(int)
    ships['time'] = ships['DATE'].apply(lambda x: ('19' + str(x)) if (str(x)[0]=='8' or str(x)[0]=='9') else ('20' + str(x))) + ships['HOUR'].apply(lambda x: str(x).zfill(2))
    ships['time'] = ships['time'].apply(lambda x: dt.datetime.strptime(x,'%Y%m%d%H'))
    
    if TargetName != 'VMAX':
        ships = ships[['YEAR', 'ID', 'time', 'VMAX', TargetName]]
    else:
        ships = ships[['YEAR', 'ID', 'time', 'VMAX']]
    if TargetName == 'POT':                 # remove outliers
        ships = ships[(ships['POT']>-9000)]
    if TargetName == 'CFLX':                # remove outliers
        ships = ships[(ships['CFLX']<5000)]
    
    ships_ID_unique = sorted(ships['ID'].unique())
    ships_processed = pd.DataFrame(columns=['YEAR', 'ID', 'time', TargetName, 'type', 'duration', 'duration_type'])
    for TC_ID in ships_ID_unique:
        TC_data = ships[ships['ID'] == TC_ID].sort_values(by='time').reset_index(drop=True)
        if len(TC_data) > 0:
            if len(TC_data[TC_data['VMAX'] >= 34]) > 0:
                start_idx = TC_data[TC_data['VMAX'] >= 34].index[0]
                TC_ID_data = TC_data.iloc[start_idx:].reset_index(drop=True)
                TC_duration = (TC_ID_data.iloc[-1]['time'] - TC_ID_data.iloc[0]['time']).total_seconds() / 3600 + 3
                if TC_duration < 100:
                    TC_duration_type = 'short'
                elif TC_duration < 200:
                    TC_duration_type = 'medium'
                else:
                    TC_duration_type = 'long'
                LMI_idx = TC_ID_data['VMAX'].idxmax()
                TC_ID_data['type'] = 'before'
                TC_ID_data.loc[LMI_idx:, 'type'] = 'after'
                TC_ID_data['duration'] = TC_duration
                TC_ID_data['duration_type'] = TC_duration_type
                TC_ID_data = TC_ID_data.dropna().reset_index(drop=True)
                ships_processed = pd.concat([ships_processed, TC_ID_data[['YEAR', 'ID', 'time', TargetName, 'type', 'duration', 'duration_type']]], axis=0, ignore_index=True)


    def predictor_trend(df, col):   
        save_folder = f'figures/SHIPS_TS_34/trends/drop_separate/{basin}'
        title_str = 'each predictor dropnan separately'

        # Target trend
        annual_mean = df.groupby('YEAR').agg({col: ['mean', 'count']}).reset_index()
        years = annual_mean['YEAR'].values.reshape(-1).astype(int)
        values = annual_mean[(col, 'mean')].values.reshape(-1).astype(float)
        num_years_all = len(years)

        model = sm.OLS(values, sm.add_constant(years)).fit()
        y_pred = model.predict(sm.add_constant(years))
        slope = model.params[1]
        intercept = model.params[0]
        r2 = model.rsquared
        p_value = model.pvalues[1]
        ci = model.conf_int()[1]
        ci_width = (ci[1] - ci[0]) / 2


        # Target trend before LMI
        annual_mean_before = df[df['type'] == 'before'].groupby('YEAR').agg({col: ['mean', 'count']}).reset_index()
        years = annual_mean_before['YEAR'].values.reshape(-1)
        values = annual_mean_before[(col, 'mean')].values.reshape(-1).astype(float)
        num_years_before = len(years)

        model_before = sm.OLS(values, sm.add_constant(years)).fit()
        y_pred_before = model_before.predict(sm.add_constant(years))
        slope_before = model_before.params[1]
        intercept_before = model_before.params[0]
        r2_before = model_before.rsquared
        p_value_before = model_before.pvalues[1]
        ci_before = model_before.conf_int()[1]
        ci_width_before = (ci_before[1] - ci_before[0]) / 2


        # Target trend after LMI
        annual_mean_after = df[df['type'] == 'after'].groupby('YEAR').agg({col: ['mean', 'count']}).reset_index()
        years = annual_mean_after['YEAR'].values.reshape(-1)
        values = annual_mean_after[(col, 'mean')].values.reshape(-1).astype(float)
        num_years_after = len(years)
        
        model_after = sm.OLS(values, sm.add_constant(years)).fit() 
        y_pred_after = model_after.predict(sm.add_constant(years))
        slope_after = model_after.params[1]
        intercept_after = model_after.params[0]
        r2_after = model_after.rsquared
        p_value_after = model_after.pvalues[1]
        ci_after = model_after.conf_int()[1]
        ci_width_after = (ci_after[1] - ci_after[0]) / 2

        return slope, p_value, ci_width, slope_before, p_value_before, ci_width_before, slope_after, p_value_after, ci_width_after, num_years_all, num_years_before, num_years_after


    csv_all = f'SHIPS_trends/{basin}/{basin}_all.csv'
    csv_short = f'SHIPS_trends/{basin}/{basin}_short.csv'
    csv_medium = f'SHIPS_trends/{basin}/{basin}_medium.csv'
    csv_long = f'SHIPS_trends/{basin}/{basin}_long.csv'
    
    if os.path.exists(csv_all):
        save_all = pd.read_csv(csv_all)
        save_short = pd.read_csv(csv_short)
        save_medium = pd.read_csv(csv_medium)
        save_long = pd.read_csv(csv_long)
    else:
        save_all = pd.DataFrame(columns=['Target', 'slope_all', 'p_value_all', 'ci_width_all', 'slope_before', 'p_value_before', 'ci_width_before', 'slope_after', 'p_value_after', 'ci_width_after', 'num_years_all', 'num_years_before', 'num_years_after'])
        save_short = pd.DataFrame(columns=['Target', 'slope_all', 'p_value_all', 'ci_width_all', 'slope_before', 'p_value_before', 'ci_width_before', 'slope_after', 'p_value_after', 'ci_width_after', 'num_years_all', 'num_years_before', 'num_years_after'])
        save_medium = pd.DataFrame(columns=['Target', 'slope_all', 'p_value_all', 'ci_width_all', 'slope_before', 'p_value_before', 'ci_width_before', 'slope_after', 'p_value_after', 'ci_width_after', 'num_years_all', 'num_years_before', 'num_years_after'])
        save_long = pd.DataFrame(columns=['Target', 'slope_all', 'p_value_all', 'ci_width_all', 'slope_before', 'p_value_before', 'ci_width_before', 'slope_after', 'p_value_after', 'ci_width_after', 'num_years_all', 'num_years_before', 'num_years_after'])
    
    # All TCs
    rgr_all = predictor_trend(ships_processed, TargetName)
    save_all.loc[len(save_all)] = [TargetName, *rgr_all]
    save_all.to_csv(csv_all, index=False)

    # Short-live TCs
    ships_short = ships_processed[ships_processed['duration_type'] == 'short']
    rgr_short = predictor_trend(ships_short, TargetName)
    save_short.loc[len(save_short)] = [TargetName, *rgr_short]
    save_short.to_csv(csv_short, index=False)

    # Medium-live TCs
    ships_medium = ships_processed[ships_processed['duration_type'] == 'medium']
    rgr_medium = predictor_trend(ships_medium, TargetName)
    save_medium.loc[len(save_medium)] = [TargetName, *rgr_medium]
    save_medium.to_csv(csv_medium, index=False)

    # Long-live TCs
    ships_long = ships_processed[ships_processed['duration_type'] == 'long']
    rgr_long = predictor_trend(ships_long, TargetName)
    save_long.loc[len(save_long)] = [TargetName, *rgr_long]
    save_long.to_csv(csv_long, index=False)



In [22]:
basins = ['EPAC', 'WPAC']
variables = ['SHRD','D200','TPW','PC2','SDBT','POT','CFLX','VMPI']
for basin in basins:
    for variable in variables:
        SHIPS_variable_trend(basin, variable)

# Table S1

In [12]:
def duration_group_comparison(basin, split_year):
    IB_data = pd.read_csv('processed_data/IBTrACS_%s_processed_dt3_202510.csv'%basin)
    IB_data['ISO_TIME'] = pd.to_datetime(IB_data['ISO_TIME'])  
    ID_unique = sorted(IB_data['USA_ATCF_ID'].unique())        

    duration_df = pd.DataFrame(columns=['year', 'ID', 'duration'])     
    for TC_ID in ID_unique:
        TC_data = IB_data[IB_data['USA_ATCF_ID'] == TC_ID].sort_values(by='ISO_TIME').reset_index(drop=True)
        if len(TC_data) > 0:
            TC_year = TC_data['YEAR'].iloc[0]
            if len(TC_data[TC_data['COMBINE_WIND'] >= 34]) > 0:
                start_idx = TC_data[TC_data['COMBINE_WIND'] >= 34].index[0]
                if len(TC_data[TC_data['NATURE'] == 'TS']) > 0:    
                    end_idx = TC_data[TC_data['NATURE'] == 'TS'].index[-1]
                    if end_idx > start_idx:
                        TC_duration = (TC_data['ISO_TIME'].iloc[end_idx] - TC_data['ISO_TIME'].iloc[start_idx]).total_seconds()/3600 + 3
                        duration_df = pd.concat([duration_df, pd.DataFrame({'year':[TC_year], 'ID':[TC_ID], 'duration':[TC_duration]})], ignore_index=True)
            
    
    bins = [0, 100, 200, float('inf')]
    labels = ['0-100h', '100-200h', '>200h']
    
    early_period = duration_df[duration_df['year'] < split_year]
    late_period = duration_df[duration_df['year'] >= split_year]

    early_types = pd.cut(early_period['duration'], bins=bins, labels=labels)
    late_types = pd.cut(late_period['duration'], bins=bins, labels=labels)

    early_counts = early_types.value_counts().sort_index()
    late_counts = late_types.value_counts().sort_index()
    
    contingency = pd.DataFrame({
        f'Occurrence Before {split_year}': early_counts,
        f'Occurrence of {split_year} and After': late_counts
    })
    
    chi2, p_value = stats.chi2_contingency(contingency)[:2]
    
    
    print(f"\nDuration Group Comparison for {basin} Basin")
    print("------------------------------------------------")
    print(contingency)
    print("\nChi-squared test results:")
    print(f"Chi-squared statistic: {chi2:.2f}")
    print(f"p-value: {p_value}")

    if p_value < 0.05:
        print("The distributions are significantly different (p < 0.05)")
    else:
        print("The distributions are not significantly different (p >= 0.05)")

duration_group_comparison('EP', 2003)
duration_group_comparison('WP', 2003)


Duration Group Comparison for EP Basin
------------------------------------------------
          Occurrence Before 2003  Occurrence of 2003 and After
duration                                                      
0-100h                       118                           177
100-200h                     143                           148
>200h                         79                            35

Chi-squared test results:
Chi-squared statistic: 28.32
p-value: 7.08561968193573e-07
The distributions are significantly different (p < 0.05)

Duration Group Comparison for WP Basin
------------------------------------------------
          Occurrence Before 2003  Occurrence of 2003 and After
duration                                                      
0-100h                       169                           187
100-200h                     252                           247
>200h                        143                            90

Chi-squared test results:
Chi-squared statistic:

# Table S3

In [18]:
def north_south_trends(basin, duration_type):
    IB_data = pd.read_csv('processed_data/IBTrACS_%s_processed_dt3_202510.csv'%basin)
    IB_data['ISO_TIME'] = pd.to_datetime(IB_data['ISO_TIME'])  
    ID_unique = sorted(IB_data['USA_ATCF_ID'].unique())        

    TC_info_df = pd.DataFrame(columns=['year', 'ID', 'duration', 'LMI', 'start_lat', 'start_lon'])     
    for TC_ID in ID_unique:
        TC_data = IB_data[IB_data['USA_ATCF_ID'] == TC_ID].sort_values(by='ISO_TIME').reset_index(drop=True)
        if len(TC_data) > 0:
            TC_year = TC_data['YEAR'].iloc[0]
            if len(TC_data[TC_data['COMBINE_WIND'] >= 34]) > 0:
                start_idx = TC_data[TC_data['COMBINE_WIND'] >= 34].index[0]
                start_lat = TC_data['LAT'].iloc[start_idx]
                start_lon = TC_data['LON'].iloc[start_idx]
                if len(TC_data[TC_data['NATURE'] == 'TS']) > 0:    
                    end_idx = TC_data[TC_data['NATURE'] == 'TS'].index[-1]
                    if end_idx > start_idx:
                        TC_duration = (TC_data['ISO_TIME'].iloc[end_idx] - TC_data['ISO_TIME'].iloc[start_idx]).total_seconds()/3600 + 3
                        TC_valid = TC_data.iloc[start_idx:end_idx+1].reset_index(drop=True)
                        if TC_duration < 100:
                            TC_type = 'short'
                        elif TC_duration < 200:
                            TC_type = 'medium'
                        else:
                            TC_type = 'long'
                        TC_info_df = pd.concat([TC_info_df, pd.DataFrame({'year':[TC_year], 'ID':[TC_ID], 'duration':[TC_duration], 'start_lat':[start_lat], 'type':[TC_type]})], ignore_index=True)

    if duration_type == 'all':
        pass
    else:
        TC_info_df = TC_info_df[TC_info_df['type'] == duration_type]

    years = sorted(TC_info_df['year'].unique())
    divide_lat = {'EP':15, 'WP':15}
    basin_lat = divide_lat[basin]
    count_north = []
    count_south = []
    duration_north = []
    duration_south = []
    for year in years:
        TC_info_df_year = TC_info_df[TC_info_df['year'] == year]
        north_year = TC_info_df_year[TC_info_df_year['start_lat'] > basin_lat]
        south_year = TC_info_df_year[TC_info_df_year['start_lat'] <= basin_lat]
        count_north.append((year, len(north_year)))
        count_south.append((year, len(south_year)))
        duration_north.append((year, np.mean(north_year['duration'])))
        duration_south.append((year, np.mean(south_year['duration'])))
    count_north = sorted(count_north, key=lambda x: x[0])
    count_south = sorted(count_south, key=lambda x: x[0])
    years_count_north, counts_north = zip(*count_north)
    years_count_south, counts_south = zip(*count_south)

    duration_north = sorted(duration_north, key=lambda x: x[0])
    duration_south = sorted(duration_south, key=lambda x: x[0])
    # Remove NaN values from duration lists
    duration_north = [(year, dur) for year, dur in duration_north if not np.isnan(dur)]
    duration_south = [(year, dur) for year, dur in duration_south if not np.isnan(dur)]
    years_duration_north, durations_north = zip(*duration_north)     
    years_duration_south, durations_south = zip(*duration_south)
    
    def trend_by_region(year_list, value_list, region, variable):
        X = sm.add_constant(year_list)
        model = sm.OLS(value_list, X).fit()
        y_pred = model.predict(X)
        slope = model.params[1]
        intercept = model.params[0]
        r2 = model.rsquared 
        p_value = model.pvalues[1]
        ci = model.conf_int()[1]
        print(f'{basin} {duration_type} {region} {variable} slope: {slope:.2f}, p-value: {p_value:.3f}, CI_width: {(ci[1]-ci[0])/2:.2f}')
        

    trend_by_region(years_count_north, counts_north, 'North', 'Occurrence')
    trend_by_region(years_count_south, counts_south, 'South', 'Occurrence')
    trend_by_region(years_duration_north, durations_north, 'North', 'Duration')
    trend_by_region(years_duration_south, durations_south, 'South', 'Duration')

types = ['all', 'short', 'medium', 'long']
for duration_type in types:
    north_south_trends('EP', duration_type)
    north_south_trends('WP', duration_type)
    print()


EP all North Occurrence slope: 0.07, p-value: 0.017, CI_width: 0.06
EP all South Occurrence slope: -0.08, p-value: 0.072, CI_width: 0.09
EP all North Duration slope: -1.29, p-value: 0.002, CI_width: 0.79
EP all South Duration slope: -1.39, p-value: 0.000, CI_width: 0.69
WP all North Occurrence slope: 0.05, p-value: 0.353, CI_width: 0.10
WP all South Occurrence slope: -0.16, p-value: 0.002, CI_width: 0.10
WP all North Duration slope: -0.55, p-value: 0.063, CI_width: 0.58
WP all South Duration slope: -0.56, p-value: 0.162, CI_width: 0.80

EP short North Occurrence slope: 0.09, p-value: 0.000, CI_width: 0.04
EP short South Occurrence slope: 0.03, p-value: 0.210, CI_width: 0.05
EP short North Duration slope: -0.55, p-value: 0.014, CI_width: 0.43
EP short South Duration slope: -0.37, p-value: 0.075, CI_width: 0.41
WP short North Occurrence slope: 0.06, p-value: 0.085, CI_width: 0.07
WP short South Occurrence slope: 0.00, p-value: 0.987, CI_width: 0.04
WP short North Duration slope: -0.18, p